## Visualize word2vec embeddings in tensorboard

In [2]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import gensim
import tensorflow as tf
import numpy as np

from tensorflow.contrib.tensorboard.plugins import projector

Load saved word2vec model to visualize.

In [3]:
fname = "w2v_model"
model = gensim.models.keyedvectors.KeyedVectors.load(fname)

# project vocab
max = len(model.wv.vocab)-1
w2v = np.zeros((max,model.layer1_size))

with open("metadata.tsv", 'w+') as file_metadata:
    for i,word in enumerate(model.wv.index2word[:max]):
        w2v[i] = model.wv[word]
        file_metadata.write(word + '\n')
        
# define the model without training
sess = tf.InteractiveSession()

with tf.device("/cpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name='embedding')

tf.global_variables_initializer().run()

path = 'tensorboard/1'
saver = tf.train.Saver()
writer = tf.summary.FileWriter(path, sess.graph)

# adding into projector
config = projector.ProjectorConfig()
embed= config.embeddings.add()
embed.tensor_name = 'embedding'
embed.metadata_path = 'metadata.tsv'

# Specify the width and height of a single thumbnail.
projector.visualize_embeddings(writer, config)

saver.save(sess, path+'/model.ckpt', global_step=max)

# open tensorboard with logdir, check localhost:6006 for viewing your embedding.
# tensorboard --logdir="tensorboard/1"

'tensorboard/1/model.ckpt-28071'